# Introduction to Model Optimization

**Module 03 | Notebook 1 of 5**

This notebook introduces the key techniques for making LLMs faster, smaller, and cheaper to deploy.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand why optimization is essential for production
2. Compare distillation, pruning, and quantization techniques
3. Choose the right optimization strategy for your use case
4. Calculate cost-benefit trade-offs

---

In [ ]:
%%capture
!pip install transformers torch matplotlib pandas

In [ ]:
import torch
from transformers import AutoModel, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

---

## 1️⃣ Why Optimize LLMs?

### The Production Challenge

```
┌─────────────────────────────────────────────────────────────────┐
│                     The LLM Trilemma                            │
│                                                                 │
│                        ACCURACY                                 │
│                           ▲                                     │
│                          / \                                    │
│                         /   \                                   │
│                        /     \                                  │
│              SPEED ◄──────────► COST                            │
│                                                                 │
│   Large models: High accuracy, slow, expensive                  │
│   Small models: Fast, cheap, but less accurate                  │
│                                                                 │
│   🎯 Goal: Get the best of all three with optimization!        │
└─────────────────────────────────────────────────────────────────┘
```

### Real-World Constraints

| Constraint | Impact |
|------------|--------|
| **Latency** | Users expect <100ms response times |
| **Memory** | Edge devices have limited RAM |
| **Cost** | GPU compute is expensive at scale |
| **Energy** | Large models consume significant power |
| **Throughput** | Need to serve thousands of requests/sec |

In [ ]:
# Compare model sizes
models = [
    ("distilbert-base-uncased", "DistilBERT"),
    ("bert-base-uncased", "BERT-base"),
    ("bert-large-uncased", "BERT-large"),
]

data = []
for model_name, label in models:
    print(f"Loading {label}...")
    model = AutoModel.from_pretrained(model_name)
    params = sum(p.numel() for p in model.parameters())
    size_mb = params * 4 / (1024**2)  # FP32
    data.append({"Model": label, "Parameters (M)": params/1e6, "Size (MB)": size_mb})
    del model

df = pd.DataFrame(data)
print("\nModel Comparison:")
print(df.to_string(index=False))

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

colors = ['#2ecc71', '#3498db', '#9b59b6']

axes[0].barh(df['Model'], df['Parameters (M)'], color=colors)
axes[0].set_xlabel('Parameters (Millions)')
axes[0].set_title('Model Size Comparison')

axes[1].barh(df['Model'], df['Size (MB)'], color=colors)
axes[1].set_xlabel('Memory (MB)')
axes[1].set_title('Memory Footprint (FP32)')

plt.tight_layout()
plt.show()

# Calculate savings
bert_params = df[df['Model']=='BERT-base']['Parameters (M)'].values[0]
distil_params = df[df['Model']=='DistilBERT']['Parameters (M)'].values[0]
print(f"\nDistilBERT is {bert_params/distil_params:.1f}x smaller than BERT-base")
print(f"That's a {(1-distil_params/bert_params)*100:.0f}% reduction!")

---

## 2️⃣ The Three Pillars of Optimization

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Model Optimization Techniques                        │
├───────────────────────┬───────────────────────┬─────────────────────────┤
│     DISTILLATION      │       PRUNING         │      QUANTIZATION       │
├───────────────────────┼───────────────────────┼─────────────────────────┤
│                       │                       │                         │
│  Teacher → Student    │  Remove connections   │  Reduce precision       │
│                       │                       │                         │
│  ┌───────────────┐    │  ●●●●●●  →  ●○●○●●   │  32-bit → 8-bit         │
│  │ Large Model   │    │  ●●●●●●     ●●○○●●   │                         │
│  │   (Teacher)   │    │  ●●●●●●     ○●●●○●   │  1.234567890            │
│  └───────┬───────┘    │                       │       ↓                 │
│          │ teaches    │                       │    1.23                 │
│          ▼            │                       │                         │
│  ┌───────────────┐    │                       │                         │
│  │ Small Model   │    │                       │                         │
│  │   (Student)   │    │                       │                         │
│  └───────────────┘    │                       │                         │
│                       │                       │                         │
├───────────────────────┼───────────────────────┼─────────────────────────┤
│ Size: 2-10x smaller   │ Size: 50-90% sparse   │ Size: 2-4x smaller      │
│ Speed: 2-3x faster    │ Speed: 1.5-3x faster  │ Speed: 2-4x faster      │
│ Accuracy: 95-99%      │ Accuracy: 90-99%      │ Accuracy: 95-99%        │
└───────────────────────┴───────────────────────┴─────────────────────────┘
```

### Technique Comparison

| Technique | How It Works | Pros | Cons |
|-----------|--------------|------|------|
| **Distillation** | Train small model to mimic large model | Most accuracy retention | Requires training |
| **Pruning** | Remove unimportant weights | No retraining needed | Sparse operations support varies |
| **Quantization** | Reduce numerical precision | Easy to apply, big speedups | Some accuracy loss |

---

## 3️⃣ Quick Demo: Quantization in Action

In [ ]:
from transformers import AutoTokenizer

# Load a small model for demo
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_fp32 = AutoModelForSequenceClassification.from_pretrained(model_name)

# Model size in FP32
params = sum(p.numel() for p in model_fp32.parameters())
fp32_size = params * 4  # 4 bytes per float32

print(f"Model: {model_name}")
print(f"Parameters: {params:,}")
print(f"FP32 Size: {fp32_size / 1024**2:.1f} MB")

In [ ]:
# Apply dynamic quantization (INT8)
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32, 
    {torch.nn.Linear},  # Quantize Linear layers
    dtype=torch.qint8
)

print("\n✅ Dynamic quantization applied!")

In [ ]:
# Compare inference speed
test_text = "This movie was absolutely fantastic! I loved every minute of it."
inputs = tokenizer(test_text, return_tensors="pt")

# Warmup
with torch.no_grad():
    _ = model_fp32(**inputs)
    _ = model_int8(**inputs)

# Benchmark FP32
n_runs = 100
start = time.time()
for _ in range(n_runs):
    with torch.no_grad():
        _ = model_fp32(**inputs)
fp32_time = (time.time() - start) / n_runs * 1000

# Benchmark INT8
start = time.time()
for _ in range(n_runs):
    with torch.no_grad():
        _ = model_int8(**inputs)
int8_time = (time.time() - start) / n_runs * 1000

print(f"Inference Latency (avg of {n_runs} runs):")
print(f"  FP32: {fp32_time:.2f} ms")
print(f"  INT8: {int8_time:.2f} ms")
print(f"  Speedup: {fp32_time/int8_time:.2f}x")

In [ ]:
# Verify outputs are similar
with torch.no_grad():
    fp32_output = model_fp32(**inputs).logits
    int8_output = model_int8(**inputs).logits

fp32_probs = torch.softmax(fp32_output, dim=-1)
int8_probs = torch.softmax(int8_output, dim=-1)

print("\nPrediction Comparison:")
print(f"  FP32: POSITIVE={fp32_probs[0,1]:.4f}, NEGATIVE={fp32_probs[0,0]:.4f}")
print(f"  INT8: POSITIVE={int8_probs[0,1]:.4f}, NEGATIVE={int8_probs[0,0]:.4f}")
print(f"  Difference: {torch.abs(fp32_probs - int8_probs).max():.6f}")

---

## 4️⃣ Decision Framework: Choosing the Right Technique

```
                    Which optimization to use?
                            │
            ┌───────────────┴───────────────┐
            │                               │
    Need significant           Just need speed/
    size reduction?            memory savings?
            │                               │
     ┌──────┴──────┐                 ┌──────┴──────┐
     │             │                 │             │
  Can train?  No training?      Accuracy      Speed is
     │             │             critical?     priority?
     │             │                 │             │
     ▼             ▼                 ▼             ▼
 DISTILLATION   PRUNING       QUANTIZATION   QUANTIZATION
 (best quality) (quick)          (INT8)         (INT4)
```

In [ ]:
# Decision helper
decision_matrix = pd.DataFrame({
    'Technique': ['Distillation', 'Pruning', 'Quantization (INT8)', 'Quantization (INT4)'],
    'Size Reduction': ['50-90%', '50-80%', '50%', '75%'],
    'Speed Improvement': ['2-4x', '1.5-3x', '2-3x', '3-4x'],
    'Accuracy Loss': ['1-5%', '1-10%', '<1%', '2-5%'],
    'Requires Training': ['Yes', 'Optional', 'No', 'No'],
    'Difficulty': ['High', 'Medium', 'Low', 'Low']
})

print("Optimization Decision Matrix:")
print("=" * 90)
print(decision_matrix.to_string(index=False))

---

## 5️⃣ Cost Analysis

Let's calculate the potential savings from optimization:

In [ ]:
def calculate_savings(requests_per_day, cost_per_1k_tokens_original, optimization_speedup):
    """
    Calculate monthly cost savings from optimization.
    """
    tokens_per_request = 500  # Average
    
    # Original cost
    daily_tokens = requests_per_day * tokens_per_request
    daily_cost_original = (daily_tokens / 1000) * cost_per_1k_tokens_original
    monthly_cost_original = daily_cost_original * 30
    
    # Optimized cost (assuming cost scales with compute)
    monthly_cost_optimized = monthly_cost_original / optimization_speedup
    
    savings = monthly_cost_original - monthly_cost_optimized
    
    return {
        'monthly_requests': requests_per_day * 30,
        'original_cost': monthly_cost_original,
        'optimized_cost': monthly_cost_optimized,
        'monthly_savings': savings,
        'annual_savings': savings * 12
    }

# Example scenarios
scenarios = [
    {"name": "Small App", "requests": 10000, "cost": 0.002, "speedup": 2},
    {"name": "Medium App", "requests": 100000, "cost": 0.002, "speedup": 2},
    {"name": "Large App", "requests": 1000000, "cost": 0.002, "speedup": 3},
]

print("Cost Savings Analysis:")
print("=" * 70)
for s in scenarios:
    result = calculate_savings(s['requests'], s['cost'], s['speedup'])
    print(f"\n{s['name']} ({s['requests']:,} requests/day, {s['speedup']}x speedup):")
    print(f"  Monthly requests: {result['monthly_requests']:,}")
    print(f"  Original cost: ${result['original_cost']:,.0f}/month")
    print(f"  Optimized cost: ${result['optimized_cost']:,.0f}/month")
    print(f"  Monthly savings: ${result['monthly_savings']:,.0f}")
    print(f"  Annual savings: ${result['annual_savings']:,.0f}")

---

## 6️⃣ Combining Techniques

For maximum optimization, techniques can be combined:

```
Original Model (BERT-large, 340M params)
            │
            ▼ Distillation
Distilled Model (DistilBERT, 66M params)  [5x smaller]
            │
            ▼ Pruning (50%)
Pruned Model (33M effective params)       [10x smaller]
            │
            ▼ Quantization (INT8)
Final Model (~16.5MB)                     [20x smaller!]
```

In [ ]:
# Visualize combined optimization
stages = ['BERT-large', 'Distilled', 'Pruned (50%)', 'Quantized (INT8)']
sizes = [340, 66, 33, 16.5]  # Approximate sizes in millions/MB
colors = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']

fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.bar(stages, sizes, color=colors, edgecolor='black')
ax.set_ylabel('Size (MB / Million params)')
ax.set_title('Combined Optimization: 20x Reduction!')

# Add reduction labels
for i, (bar, size) in enumerate(zip(bars, sizes)):
    if i > 0:
        reduction = (1 - size/sizes[0]) * 100
        ax.annotate(f'-{reduction:.0f}%', 
                   xy=(bar.get_x() + bar.get_width()/2, size + 10),
                   ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

---

## 🎯 Student Challenge

### Challenge: Calculate Your Optimization Strategy

In [ ]:
# TODO: Given these requirements, recommend an optimization strategy:

# Scenario:
# - Current model: BERT-large (340M params)
# - Target latency: <50ms (currently 200ms)
# - Accuracy tolerance: Can accept up to 3% accuracy drop
# - Training budget: No GPU hours available for training
# - Deployment target: Cloud GPU

# Questions:
# 1. Which technique(s) would you recommend?
# 2. What size reduction could you achieve?
# 3. How much speedup would you expect?

# Your answer:


---

## 📝 Key Takeaways

1. **Optimization is essential** for production deployments
2. **Three main techniques**: Distillation, Pruning, Quantization
3. **Trade-offs exist** between size, speed, and accuracy
4. **Techniques can be combined** for maximum benefit
5. **Cost savings** can be substantial at scale

---

## ➡️ Next Steps

Continue to `02_knowledge_distillation.ipynb` for hands-on distillation!